In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import world_bank_data as wb
import xml.etree.ElementTree as ET
#import quandl
#from IPython.display import Image

# Estimate True Size of China's Economy 
By Joeri Schasfoort, PhD & Carlo Humpert, Msc. 

Based on Martinez (2022)

https://www.journals.uchicago.edu/doi/10.1086/720458

## GDP data collection from World Bank

In [84]:
China_GDP_LCU = pd.DataFrame(wb.get_series('NY.GDP.MKTP.KN', country='CHN', simplify_index=True))
China_GDP_USD2015 = pd.DataFrame(wb.get_series('NY.GDP.MKTP.KD', country='CHN', simplify_index=True))
China_GDP_PPP2017 = pd.DataFrame(wb.get_series('NY.GDP.MKTP.PP.KD', country='CHN', simplify_index=True))
USA_GDP_PPP2017 = pd.DataFrame(wb.get_series('NY.GDP.MKTP.PP.KD', country='USA', simplify_index=True))
USA_GDP_USD2015 = pd.DataFrame(wb.get_series('NY.GDP.MKTP.KD', country='USA', simplify_index=True))

Then, we collect them for the relevant years

In [87]:
China_GDP_CLCU_1992 = int(China_GDP_LCU.loc['1992'])
print('China GDP LCU 1992 = ', China_GDP_CLCU_1992)
China_GDP_CLCU_2021 = int(China_GDP_LCU.loc['2021'])
print('China GDP LCU 2021 = ', China_GDP_CLCU_2021)
China_GDP_USD2015_2021 = int(China_GDP_USD2015.loc['2021'])
print('China GDP C2015-USD 2021 = ', China_GDP_USD2015_2021)
China_GDP_PPP2017_2021 = int(China_GDP_PPP2017.loc['2021'])
print('China GDP C2017-PPP 2021 = ', China_GDP_PPP2017_2021)
USA_GDP_USD2015_2021 = int(USA_GDP_USD2015.loc['2021'])
print('USA GDP C2015-USD 2021 = ', USA_GDP_USD2015_2021)
USA_GDP_PPP2017_2021 = int(USA_GDP_PPP2017.loc['2021'])
print('USA GDP C2015-USD 2021 = ', USA_GDP_PPP2017_2021)

China GDP LCU 1992 =  7985009112300
China GDP LCU 2021 =  98406400602000
China GDP C2015-USD 2021 =  15801910975833
China GDP C2017-PPP 2021 =  24861343804323
USA GDP C2015-USD 2021 =  20338578000000
USA GDP C2015-USD 2021 =  20932281461072


## Step by step calculation

### 1) Translate GDP to Log GDP (because this is the scale Martinez uses)

GDP 1992 in natural logarithm: 29.708587 

GDP 2021 in natural logarithm: 32.220127

In [20]:
log_GDP_1992 = np.log(China_GDP_CLCU_1992)
log_GDP_1992

29.70858703877208

In [21]:
log_GDP_2021 = np.log(China_GDP_CLCU_2021)
log_GDP_2021

32.2201269646396

### 2) Calculate China's Reported Average GDP Growth (continuous & annual)

Subtract the log GDP 1992 from the log GDP 2021

32.220127 - 29.708587 = 2.51154 

As it is the difference in natural logarithms, it represents the continuous growth rate. The yearly continuous growth rate can be then calculated simply by dividing through the number of years: 2.51154/29 = 0.0866 = 8.66%

In [66]:
observed_years = (2021 - 1991)
log_difference = log_GDP_2021 - log_GDP_1992
continuous_growth = log_difference / observed_years
continuous_growth

0.08371799752891723

To calculate the **annual growth rate**, we take e to the power of the yearly continuous growth rate

e^0.0866 - 1 = 0.0905 = 9.05%

This is the official annual growth rate according to the official statistics.

In [67]:
annual_growth = np.exp(continuous_growth) - 1
annual_growth

0.08732222301614412

### 3) Calculate Martinez's China Growth Deflator 

For our estimated growth deflator, we use the same estimate as Martínez himself, which can be found in column 7 in Table 1 in: H[ow Much Should We Trust the Dictator’s GDP Growth Estimates?](https://www.journals.uchicago.edu/doi/abs/10.1086/720458) by Luis R. Martínez Journal of Political Economy 2022 130:10, 2731-2769

For his calculations, Martínez relies on the Freedom in the World (FiW) index, see [Freedom House](https://freedomhouse.org/reports/publication-archives) [Publication Achieves](https://freedomhouse.org/sites/default/files/2022-03/Country_and_Territory_Ratings_and_Statuses_FIW_1973-2022%20.xlsx). (”[Prior to the 2020 edition](https://freedomhouse.org/reports/freedom-world/freedom-world-research-methodology). *Freedom in the World* assigned a country or territory two ratings—one for political rights and one for civil liberties—based on its total scores for the political rights and civil liberties questions. Each rating of 1 to 7, with 1 representing the greatest degree of freedom and 7 the smallest degree of freedom, corresponded to a specific range of total scores. The average of the ratings determined the status of Free, Partly Free, or Not Free. While the underlying formula for converting scores into status remains identical, starting in the 2020 edition *Freedom in the World* no longer presented the 1–7 ratings as a separate element of its findings. The ratings are still included in the raw data available for download.”

The Chinese average value of the FiW index between 1992 and 2021 is 6.6; 

In [89]:
CHN_av_FiW = 6.6

Next, we calculate the effect of a 1 unit increase in the FiW index on the long-run GDP growth rate by dividing the coefficient estimate of the interaction term ( ln(NTL) x FiW ) by the coefficient estimate of the night-time light  ( ln(NTL) ) itself.

0.032 / 0.288 = 0.11

In [90]:
interaction_term_estimate = 0.032
night_time_light_estimate = 0.288

LR_GDP_overstatement = round(interaction_term_estimate / night_time_light_estimate, 2) # we round here to two digits following Martinez
LR_GDP_overstatement

0.11

This implies that “a 1 unit increase in the FiW index is associated with a long-run GDP growth rate that is overstated by a factor of 0.11” (p. 2761, footnote 33).

Next, we multiply this factor by the Chinese average value of the (adjusted) FiW index (The author adjusts the values of the FiW index by subtracting 1, to change the range of 1 to 7 into 0 to 6).

In [91]:
China_Deflator = (CHN_av_FiW - 1) * LR_GDP_overstatement + 1
China_Deflator

1.616

### 4) Estimate China's GDP 'True' Growth 

Next, we need to deflate the log change in GDP by the deflator reported by Martinez (2022) (See “Calculation of the Deflator” to understand how to get to the number 1.605)

2.51154 / 1.605 = 1.56482 (**TODO REMOVE**)

In [92]:
deflated_log_difference = log_difference / China_Deflator
deflated_log_difference

1.5541707462051464

This implies a unbiased continuous growth rate of **1.56482/29 = 0.05396= 5.4**

In [93]:
unbiased_continuous_growth = deflated_log_difference / observed_years
unbiased_continuous_growth

0.05180569154017155

And a unbiased annual growth rate of **e^0.05396 - 1 = 0.0554 = 5.54%**

In [94]:
unbiased_annual_growth = np.exp(unbiased_continuous_growth) - 1
unbiased_annual_growth

0.05317108257943848

### 5) Use estimated GDP Growth to recalculate China's 

### 5-A) Estimated Current GDP LCU

To calculate the true Chinese GDP in 2021 multiply the official GDP in 1992 by e to the power of the true log change in GDP

GDP 2021 = GDP1992 * e^1.56482= 38,182,829,416,399

China’s actual GDP in constant LCU is around **38,182,830,000,000**

In [95]:
China_Estimated_GDP_CLCU_2021 = China_GDP_CLCU_1992 * np.exp(deflated_log_difference)
China_Estimated_GDP_CLCU_2021

37778368202768.77

As a sanity check, we sum up the official log GDP of 1992 and the calculated true change in log GDP and then, calculate the unbiased Chinese GDP in 2021 by taking e to the power of the calculated  log GDP 2021.

In [96]:
np.exp(log_GDP_1992 + deflated_log_difference)

37778368202768.73

As a percentage of official GDP, that is: 

In [102]:
true_relative_GDP = China_Estimated_GDP_CLCU_2021 / China_GDP_CLCU_2021
true_relative_GDP

0.3839015345715324

### 5-B Estimated GDP USD

In [98]:
China_Estimated_GDP_USD2015_2021 = China_GDP_USD2015_2021 * true_relative_GDP
China_Estimated_GDP_USD2015_2021

6066377872785.03

In [99]:
China_USD_Share_USA = China_Estimated_GDP_USD2015_2021 / USA_GDP_USD2015_2021
China_USD_Share_USA

0.29826951878273056

### 5-C Estimated GDP PPP

In [100]:
China_Estimated_GDP_PPP2017_2021 = China_GDP_PPP2017_2021 * true_relative_GDP
China_Estimated_GDP_PPP2017_2021

9544308037990.059

In [101]:
China_PPP_Share_USA = China_Estimated_GDP_PPP2017_2021 / USA_GDP_PPP2017_2021
China_PPP_Share_USA

0.4559611935153708

# Conclusion